# Feature performance comparison

The purpose of this notebook is to compare the classification performance of the individual features, and their combination, for E. Coli

# Imports

In [19]:
import os
import sys
from IPython.display import display

sys.path.append('../src')
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC, LinearSVC
from sklearn.feature_selection import SelectKBest, VarianceThreshold
from sklearn.decomposition import PCA, KernelPCA
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.feature_selection import SelectKBest, RFE
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.base import clone

import matplotlib.pyplot as plt

from yellowbrick.features import ParallelCoordinates
from yellowbrick.features import Rank1D, Rank2D

import pandas as pd
import numpy as np
import seaborn as sns

from dataset.transporter_dataset import create_dataset
from dataset.cluster_fasta import cd_hit
from features.labels import fasta_to_labels
from features.compositions import calculate_composition_feature
from features.pssm import calculate_pssm_feature
from features.coexp import calculate_coexp_feature
from models.eval import nested_crossval
from visualization.feature_plots import create_plot

# Globals

In [20]:
N_THREADS = 16
IDENTITY_THRESHOLD=70

LOG_FILE = "../logs/meta_amino_sugar.log"
N_THREADS = 16
ORGANISM = "meta_euk"

# Dataset

In [21]:
# Delete previous log
if os.path.exists(LOG_FILE):
    with open(LOG_FILE, "w"):
        pass
# e coli, a thaliana, human
create_dataset(
    keywords_substrate_filter=["Amino-acid transport", "Sugar transport"],
    keywords_component_filter=["Transmembrane"],
    keywords_transport_filter=["Transport"],
    input_file="../data/raw/swissprot/uniprot-reviewed_yes.tab.gz",
    multi_substrate="integrate",
    verbose=True,
    outliers=["P76773", "Q47706", "P64550", "P02943", "P75733", "P69856"]
    + ["O81775", "Q9SW07", "Q9FHH5", "Q8S8A0", "Q3E965", "Q3EAV6", "Q3E8L0"]
    + ["Q9HBR0", "Q07837"],
    tax_ids_filter=[3702, 9606, 559292],
    output_tsv=f"../data/datasets/{ORGANISM}_amino_sugar.tsv",
    output_fasta=f"../data/datasets/{ORGANISM}_amino_sugar.fasta",
    output_log=LOG_FILE,
)


## Clustering

In [22]:
cd_hit(
    executable_location="cd-hit",
    input_fasta=f"../data/datasets/{ORGANISM}_amino_sugar.fasta",
    output_fasta=f"../data/datasets/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}.fasta",
    log_file=LOG_FILE,
    identity_threshold=IDENTITY_THRESHOLD,
    n_threads=N_THREADS,
    memory=4096,
    verbose=True,
)

## Annotations

In [23]:
df_annotations = pd.read_table(f"../data/datasets/{ORGANISM}_amino_sugar.tsv", index_col=0)
df_annotations.head()

,keywords_transport,keywords_location,keywords_transport_related,gene_names,protein_names,tcdb_id,organism_id,sequence
Uniprot,,,,,,,,
P0CD99,Sugar transport,Cell membrane;Membrane;Transmembrane,Transport,MPH2 YDL247W,Alpha-glucosides permease MPH2 (Maltose transp...,NaN,559292,MKNLSFLINRRKENTSDSNVYPGKAKSHEPSWIEMDDQTKKDGLDI...
Q9SFG0,Sugar transport,Membrane;Transmembrane,Symport;Transport,STP6 At3g05960 F2O10.8,Sugar transport protein 6 (Hexose transporter 6),2.A.1.1.56,3702,MAVVVSNANAPAFEAKMTVYVFICVMIAAVGGLIFGYDIGISGGVS...
Q9BWM7,Amino-acid transport,Membrane;Mitochondrion;Transmembrane,Transport,SFXN3,Sideroflexin-3,NaN,9606,MGELPLDINIQEPRWDQSTFLGRARHFFTVTDPRNLLLSGAQLEAS...
Q9ZVK6,Sugar transport,Cell membrane;Membrane;Transmembrane,Symport;Transport,SUC8 At2g14670 T6B13.9,Sucrose transport protein SUC8 (Sucrose permea...,NaN,3702,MSDLQAKNDVVAVDRQSSSSLADLDGPSPLRKMISVASIAAGIQFG...
Q08986,Amino-acid transport,Endoplasmic reticulum;Membrane;Transmembrane,Transport,SAM3 YPL274W,S-adenosylmethionine permease SAM3 (S-adenosyl...,2.A.3.10.15,559292,MDILKRGNESDKFTKIETESTTIPNDSDRSGSLIRRMKDSFKQSNL...


# Feature generation

## Labels

In [24]:
fasta_to_labels(
    input_fasta=f"../data/datasets/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}.fasta",
    output_tsv=f"../data/features/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}_labels.tsv",
)
df_labels = pd.read_table(
    f"../data/features/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}_labels.tsv",
    index_col=0,
)
df_labels.labels.value_counts()

Sugar transport         134
Amino-acid transport    115
Name: labels, dtype: int64

## PAAC

In [25]:
calculate_composition_feature(
    input_fasta=f"../data/datasets/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}.fasta",
    output_tsv=f"../data/features/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}_paac.tsv",
    feature_type="paac",
)

df_paac = pd.read_table(
    f"../data/features/{ORGANISM}_amino_sugar_cluster{IDENTITY_THRESHOLD}_paac.tsv",
    index_col=0,
)

# Functions

In [26]:
def get_feature_stats(df_features, df_labels_, labels=["Amino-acid transport", "Sugar transport"]):
    df_stats = pd.concat(
        {
            "corr": df_features.corrwith(
                df_labels_.labels.transform(lambda x: 1.0 if x == labels[1] else 0.0)
            ),
            "mean": df_features.mean(),
            "std": df_features.std(),
        },
        axis=1,
    )

    df_stats["corr_abs"] = df_stats["corr"].abs()

    df_stats["mean0"] = df_features.loc[df_labels_[df_labels_.labels == labels[0]].index].mean()
    df_stats["mean1"] = df_features.loc[df_labels_[df_labels_.labels == labels[1]].index].mean()

    df_stats["median0"] = df_features.loc[
        df_labels_[df_labels_.labels == labels[0]].index
    ].median()
    df_stats["median1"] = df_features.loc[
        df_labels_[df_labels_.labels == labels[1]].index
    ].median()

    df_stats["mediandiff"] = (df_stats["median0"] - df_stats["median1"]).abs()
    df_stats = df_stats.sort_values("mediandiff", ascending=False)
    return df_stats

In [27]:
def get_independent_test_set(
    df_features, df_labels_, labels=["Amino-acid transport", "Sugar transport"], test_size=0.2
):
    X = df_features.to_numpy()
    y = np.where(df_labels_.labels == labels[1], 1, 0)
    feature_names = df_features.columns.to_numpy()
    sample_names = df_features.index.to_numpy()
    (
        X_train,
        X_test,
        y_train,
        y_test,
        sample_names_train,
        sample_names_test,
    ) = train_test_split(
        X, y, sample_names, stratify=y, random_state=42, shuffle=True, test_size=test_size
    )
    return (
        X_train,
        X_test,
        y_train,
        y_test,
        sample_names_train,
        sample_names_test,
        feature_names,
    )


In [28]:
def print_validation_results(y_true_, y_pred_, labels = ["Amino", "Sugar"]):
    report_dict = classification_report(y_true=y_true_, y_pred=y_pred_, output_dict=True)
    report_dict = {
        labels[0]: report_dict['0'],
        labels[1]: report_dict['1'],
        "Macro": report_dict["macro avg"],
        "Weighted": report_dict["weighted avg"]
    }
    report_df = pd.DataFrame.from_dict(report_dict)
    confusion_matrix_df = pd.DataFrame(
        confusion_matrix(y_true_, y_pred_),
        columns=labels,
        index=labels,
    )
    return report_df, confusion_matrix_df

# Individual Features

## PAAC

### Stats

In [29]:
df_stats = get_feature_stats(df_paac, df_labels).head(10)
# 10 best features
display(df_stats.head(10))

display(df_stats.sort_values("corr", ascending=False))

,corr,mean,std,corr_abs,mean0,mean1,median0,median1,mediandiff
PG,-0.216319,0.002380,0.002514,0.216319,0.002966,0.001877,0.002597,0.000000,0.002597
LP,-0.313959,0.005673,0.003295,0.313959,0.006788,0.004717,0.006536,0.004228,0.002307
VS,0.255884,0.006854,0.003798,0.255884,0.005807,0.007752,0.005607,0.007901,0.002293
II,-0.266857,0.005856,0.004472,0.266857,0.007142,0.004753,0.006319,0.004086,0.002233
SL,0.180322,0.011000,0.004895,0.180322,0.010049,0.011816,0.009569,0.011802,0.002233
RF,-0.260769,0.001913,0.002052,0.260769,0.002490,0.001419,0.002222,0.000000,0.002222
LF,0.212192,0.008238,0.004508,0.212192,0.007208,0.009123,0.006791,0.008902,0.002111
MF,0.260279,0.001694,0.001840,0.260279,0.001178,0.002137,0.000000,0.002016,0.002016
ED,-0.181640,0.001944,0.002158,0.181640,0.002366,0.001582,0.002000,0.000000,0.002000
QQ,0.235629,0.001477,0.001913,0.235629,0.000991,0.001893,0.000000,0.001931,0.001931


,corr,mean,std,corr_abs,mean0,mean1,median0,median1,mediandiff
MF,0.260279,0.001694,0.001840,0.260279,0.001178,0.002137,0.000000,0.002016,0.002016
VS,0.255884,0.006854,0.003798,0.255884,0.005807,0.007752,0.005607,0.007901,0.002293
QQ,0.235629,0.001477,0.001913,0.235629,0.000991,0.001893,0.000000,0.001931,0.001931
LF,0.212192,0.008238,0.004508,0.212192,0.007208,0.009123,0.006791,0.008902,0.002111
SL,0.180322,0.011000,0.004895,0.180322,0.010049,0.011816,0.009569,0.011802,0.002233
ED,-0.181640,0.001944,0.002158,0.181640,0.002366,0.001582,0.002000,0.000000,0.002000
PG,-0.216319,0.002380,0.002514,0.216319,0.002966,0.001877,0.002597,0.000000,0.002597
RF,-0.260769,0.001913,0.002052,0.260769,0.002490,0.001419,0.002222,0.000000,0.002222
II,-0.266857,0.005856,0.004472,0.266857,0.007142,0.004753,0.006319,0.004086,0.002233
LP,-0.313959,0.005673,0.003295,0.313959,0.006788,0.004717,0.006536,0.004228,0.002307


### Independent test set

In [30]:
(
    X_train,
    X_test,
    y_train,
    y_test,
    sample_names_train,
    sample_names_test,
    feature_names,
) = get_independent_test_set(df_paac, df_labels)

### Model selection

In [31]:
for estimator in [
    LinearSVC(),
    LinearSVC(class_weight="balanced"),
    SVC(),
    SVC(class_weight="balanced"),
    GaussianNB(),
    KNeighborsClassifier(),
    RandomForestClassifier(),
    SGDClassifier(),
]:
    pipe = make_pipeline(StandardScaler(), estimator)
    scores = cross_val_score(estimator, X_train, y_train, scoring="f1_macro")
    print("### ", type(estimator).__name__)
    print(f"CV folds: {scores.round(3)}")
    print(f"Mean: {scores.mean().round(3)}")
    print(f"Std: {scores.std().round(3)}")


###  LinearSVC
CV folds: [0.344 0.344 0.355 0.355 0.35 ]
Mean: 0.35
Std: 0.005
###  LinearSVC
CV folds: [0.824 0.75  0.749 0.8   0.663]
Mean: 0.757
Std: 0.055
###  SVC
CV folds: [0.822 0.749 0.822 0.848 0.816]
Mean: 0.812
Std: 0.033
###  SVC
CV folds: [0.875 0.8   0.848 0.824 0.816]
Mean: 0.833
Std: 0.026
###  GaussianNB
CV folds: [0.824 0.749 0.774 0.749 0.795]
Mean: 0.778
Std: 0.028
###  KNeighborsClassifier
CV folds: [0.716 0.531 0.649 0.642 0.711]
Mean: 0.65
Std: 0.067
###  RandomForestClassifier
CV folds: [0.873 0.75  0.9   0.795 0.845]
Mean: 0.833
Std: 0.054
###  SGDClassifier
CV folds: [0.344 0.322 0.355 0.355 0.316]
Mean: 0.338
Std: 0.016


### Hyperparameters

In [32]:
# params = {
#     "kneighborsclassifier__n_neighbors": list(range(3,10)),
#     "kneighborsclassifier__weights": ["uniform", "distance"],
#     "kneighborsclassifier__metric": ["minkowski", "euclidean", "manhattan"],
# }
params = {
    "svc__C": [1, 10,100],
    "svc__gamma": ["scale", "auto", 1e-0, 1e-1, 1e-2],
    "svc__class_weight": ["balanced", None],
}
gsearch = GridSearchCV(
    make_pipeline(StandardScaler(), SVC()),
    param_grid=params,
    cv=5,
    scoring="f1_macro",
    n_jobs=-1,
    return_train_score=True,
)
gsearch.fit(X_train, y_train)
print(gsearch.best_params_)
print(gsearch.best_score_)
best_estimator_svc = gsearch.best_estimator_


{'svc__C': 1, 'svc__class_weight': 'balanced', 'svc__gamma': 'scale'}
0.8518482104218565


### Dimensionality reduction

In [37]:
pca = PCA()
pca.fit(X_train)
csum = np.cumsum(pca.explained_variance_ratio_)
print("Number of components to explain 95% variance:", np.argmax(csum >= 0.95) + 1)

gsearch = GridSearchCV(
    estimator=Pipeline(
        [
            ("scale", StandardScaler()),
            ("pca", PCA()),
            ("scale2", StandardScaler()),
            ("svc", SVC()),
        ]
    ),
    param_grid={
        "svc__C": [1, 10, 100],
        "svc__gamma": ["scale",1e-1, 1e-2, 1e-3],
        "svc__class_weight": ["balanced", None],
        "pca__n_components": np.linspace(0.8, 0.99, 18),
    },
    cv=5,
    scoring="f1_macro",
    n_jobs=-2,
    return_train_score=True,
)
gsearch.fit(X_train, y_train)
print(gsearch.best_params_)
print(gsearch.best_score_.round(3))
best_estimator_svc_pca = gsearch.best_estimator_


Number of components to explain 95% variance: 131
{'pca__n_components': 0.8894117647058823, 'svc__C': 10, 'svc__class_weight': None, 'svc__gamma': 0.001}
0.839


In [38]:

gsearch = GridSearchCV(
    estimator=Pipeline(
        [
            ("scale", StandardScaler()),
            ("kbest", SelectKBest()),
            ("scale2", StandardScaler()),
            ("svc", SVC()),
        ]
    ),
    param_grid={
        "svc__C": [1, 10, 100],
        "svc__gamma": [1e-1, 1e-2, 1e-3,"scale"],
        "svc__class_weight": ["balanced"],
        "kbest__k": range(1,400, 5),
    },
    cv=5,
    scoring="f1_macro",
    n_jobs=-2,
    return_train_score=True,
)
gsearch.fit(X_train, y_train)
print(gsearch.best_params_)
print(gsearch.best_score_.round(3))
best_estimator_svc_kbest = gsearch.best_estimator_


{'kbest__k': 141, 'svc__C': 1, 'svc__class_weight': 'balanced', 'svc__gamma': 'scale'}
0.868


### Validation

In [39]:
best_estimator = best_estimator_svc_pca

In [40]:
best_scores = cross_val_score(
    estimator=clone(best_estimator), X=X_train, y=y_train, scoring="f1_macro"
)
print(f"Train scores: {best_scores.mean().round(3)}+-{best_scores.std().round(3)}")

y_pred = best_estimator.predict(X_test)
y_true = y_test.copy()

report_df, confusion_matrix_df = print_validation_results(y_true, y_pred, labels=["Amino", "Sugar"])
display(report_df.round(3))
display(confusion_matrix_df)

Train scores: 0.839+-0.034


,Amino,Sugar,Macro,Weighted
precision,0.727,0.750,0.739,0.740
recall,0.696,0.778,0.737,0.740
f1-score,0.711,0.764,0.737,0.739
support,23.000,27.000,50.000,50.000


,Amino,Sugar
Amino,16,7
Sugar,6,21
